In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='optumscope'), SecretScope(name='optumscope1')]

In [0]:
dbutils.secrets.list(scope = "optumscope1")

[SecretMetadata(key='optumkeysstore')]

In [0]:
spark.conf.set(
    "fs.azure.account.key.ksrdatadlsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="optumscope1", key="optumkeysstore"))

In [0]:
display(dbutils.fs.ls("abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData"))
                      

path,name,size,modificationTime
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/Claims.csv,Claims.csv,5766,1715061116000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/Hospital.csv,Hospital.csv,1528,1715061121000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/Patient_records.csv,Patient_records.csv,5110,1715018546000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/disease.csv,disease.csv,1489,1715018546000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/group.csv,group.csv,4390,1715018546000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/subgroup.csv,subgroup.csv,561,1715018546000
abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/subscriber.csv,subscriber.csv,12061,1715018546000


In [0]:
Group = spark.read.csv("abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/group.csv",header=True, inferSchema=True,escape='"')

In [0]:
Group.show(5,False)

+-------+---------------+--------+------+----------------------------------------+--------+------+
|country|premium_written|zip_code|grp_id|grp_name                                |grp_type|city  |
+-------+---------------+--------+------+----------------------------------------+--------+------+
|India  |72000          |482018  |GRP101|Life Insurance Corporation of India     |Govt.   |Mumbai|
|India  |45000          |482049  |GRP102|HDFC Standard Life Insurance Co. Ltd.   |Private |Mumbai|
|India  |64000          |482030  |GRP103|Max Life Insurance Co. Ltd.             |Private |Delhi |
|India  |59000          |482028  |GRP104|ICICI Prudential Life Insurance Co. Ltd.|Private |Mumbai|
|India  |37000          |482014  |GRP105|Kotak Mahindra Life Insurance Co. Ltd.  |Private |Mumbai|
+-------+---------------+--------+------+----------------------------------------+--------+------+
only showing top 5 rows



In [0]:
# to find the Data types columns present in the table
Group.printSchema()

root
 |-- country: string (nullable = true)
 |-- premium_written: integer (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- grp_id: string (nullable = true)
 |-- grp_name: string (nullable = true)
 |-- grp_type: string (nullable = true)
 |-- city: string (nullable = true)



In [0]:
# to find Rows count and Cloumn Count
Group.count(),len(Group.columns)

(58, 7)

In [0]:
# drop duplicates
Group = Group.dropDuplicates()

In [0]:
# to verify Rows count and Cloumn Count
Group.count(),len(Group.columns)

(58, 7)

In [0]:
# To find null values
Group.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in Group.columns]).show()

+-------+---------------+--------+------+--------+--------+----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|
+-------+---------------+--------+------+--------+--------+----+
|      0|              0|       0|     0|       0|       0|   0|
+-------+---------------+--------+------+--------+--------+----+



In [0]:
# find distinct values
Group.select("*").distinct().show(truncate = False)

+-------+---------------+--------+------+-------------------------------------------------------------+--------+---------+
|country|premium_written|zip_code|grp_id|grp_name                                                     |grp_type|city     |
+-------+---------------+--------+------+-------------------------------------------------------------+--------+---------+
|India  |60000          |482012  |GRP142|Liberty General Insurance                                    |Private |Mumbai   |
|India  |93000          |482046  |GRP133|DHFL General Insurance                                       |Private |Mumbai   |
|India  |39000          |482024  |GRP135|Edelweiss General Insurance                                  |Private |Mumbai   |
|India  |86000          |482004  |GRP153|Shriram General Insurance                                    |Private |Jaipur   |
|India  |95000          |482003  |GRP154|Star Health and Allied Insurance                             |Private |Chennai  |
|India  |71000  

In [0]:
# to replace New Delhi with Delhi in city column
Group = Group.replace(["New Delhi"], "Delhi","city")

In [0]:
from pyspark.sql.functions import col, asc

def analyze_columns(df):
    columns = df.columns
    for column in columns:
        distinct_count = df.select(column).distinct().count()
        null_count = df.filter(col(column).isNull()).count()
        numeric_count = df.filter(col(column).rlike("^[0-9]")).count()
        text_count = df.filter(col(column).rlike("^[A-Za-z]")).count()
        special_char_count = df.filter(~col(column).rlike("^[A-Za-z0-9]")).count()
        contains_string = df.filter(col(column).rlike("[A-Za-z]")).count()
        contains_number = df.filter(col(column).rlike("[0-9]")).count()
        contains_spl_char = df.filter(col(column).rlike("[^A-Za-z0-9]")).count()

        print("Distinct Count of {} column: {}".format(column, distinct_count))
        print("Null values Count of {} column: {}\n".format(column, null_count))
        print("{} Column Start With Number: {}".format(column, numeric_count))
        print("{} Column Start With String: {}".format(column, text_count))
        print("{} Column Start With Spl.Cha: {}".format(column, special_char_count))
        print("{} Column contains string values: {}".format(column, contains_string))
        print("{} Column contains number values: {}".format(column, contains_number))
        print("{} Column contains Spl.Char values: {}\n".format(column, contains_spl_char))
        
# Example usage:
analyze_columns(Group)


Distinct Count of country column: 1
Null values Count of country column: 0

country Column Start With Number: 0
country Column Start With String: 58
country Column Start With Spl.Cha: 0
country Column contains string values: 58
country Column contains number values: 0
country Column contains Spl.Char values: 0

Distinct Count of premium_written column: 39
Null values Count of premium_written column: 0

premium_written Column Start With Number: 58
premium_written Column Start With String: 0
premium_written Column Start With Spl.Cha: 0
premium_written Column contains string values: 0
premium_written Column contains number values: 58
premium_written Column contains Spl.Char values: 0

Distinct Count of zip_code column: 36
Null values Count of zip_code column: 0

zip_code Column Start With Number: 58
zip_code Column Start With String: 0
zip_code Column Start With Spl.Cha: 0
zip_code Column contains string values: 0
zip_code Column contains number values: 58
zip_code Column contains Spl.Cha

In [0]:
Group.filter(length(col("zip_code")) < 6).count()

0

In [0]:
Group.filter(length(col("zip_code")) == 6).count()

58

In [0]:
# Define the output container path
output_container_path = "abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/StatgingData"

# Write the DataFrame to the output container path
Group.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_container_path)

# List files in the output container path
files = dbutils.fs.ls(output_container_path)

# Filter out the output file
output_file = [x for x in files if x.name.startswith("part-")]

# Move the output file to a specific location
dbutils.fs.mv(output_file[0].path, "%s/stg_Group.csv" % output_container_path)


True